# 3. CoLLiE 的并行策略

> 3.1 &ensp; 数据并行（Data Parallel）
> 
> 3.2 &ensp; 流水线并行（Pipeline Parallel）
> 
> 3.3 &ensp; 张量并行（Tensor Parallel）
> 
> 3.4 &ensp; 整合：3D并行（3D Parallel）
> 
> 3.5 &ensp; 零冗余优化器（ZeRO）

&ensp; &ensp; 本文将详细介绍CoLLiE使用的并行策略，并行策略是让模型训练突破单张显卡计算和存储上限的不同方法，CoLLiE支持的并行策略包括：数据并行 DP、流水线并行 PP、张量并行 TP、3D并行（对上述三者的整合）、零冗余优化器 ZeRO，其简要特征如下表所示。在本文中，我们将从前向传播、反向传播、代码实现、策略特点、开销计算等角度，对这些并行策略展开详细介绍。

| 名称 | 简称 | 切分数据 | 切分模型 | 切分梯度 | 切分优化器状态 | 补充说明 |
|:----|:----|:----:|:----:|:----:|:----:|:----|
| 数据并行 | DP | ✔ | ✘ | ✘ | ✘ | 每个卡保留全部的模型参数、回传梯度、优化器状态 |
| 流水线并行 | PP | ✘ | 横切 | — | — | 每张卡保留模型的不同层，存在 bubble time 的问题 | 
| 张量并行 | TP | ✘ | 竖切 | — | — | 每张卡保留每层的一部分，注意 先列切分 再行切分 |
| （整合）| 3D | ✔ | 切两次 | — | — | 结构上，先PP，再TP，通过在每dp_size张卡保留相同切分，实现DP |
| 零冗余优化器 | ZeRO1 | ✔ | ✘ | ✘ | ✔ | 在DP基础上，每张卡保留不同参数的优化器状态，直接切，均匀分 |
|  | ZeRO2 | ✔ | ✘ | ✔ | ✔ | 在ZeRO1基础上切分梯度，每张卡保留不同参数的优化器状态和对应梯度 | 
|  | ZeRO3 | ✔ | ✔ | ✔ | ✔ | 在ZeRO2基础上切分模型，每张卡保留不同参数，并且处理不同数据 |

## 3.1 &ensp; 数据并行（Data Parallel）

&ensp; &ensp; 数据并行（Data Parallel，简称 DP）是最简单的并行方案。严格来说，DP只能说实现了分布式训练，并不能真的实现模型并行；虽然提升了模型训练效率，但是在存储上存在巨大的冗余，无法解决单张卡装不下整个大模型的问题。

&emsp; **前向传播** &ensp; 如下左图（或右图）所示，假设我们有1个大语言模型，基于Transformer架构，包括N层自注意力模块（蓝色）、输入层和输出层（红色），也有1台服务器上的K张显卡。DP首先**让每张卡保留相同的完整的模型**，接着在每次前向传播时，**将不同batch的数据分配给不同的卡**；如下左图所示，K张显卡同时计算K个batch，每张显卡彼此之间独立计算，没有通信与同步。

&emsp; **反向传播** &ensp; 如下右图所示，前向传播之后，每张卡上产生了对应数据的损失。接着，和前向传播相同，每张卡对各自的损失进行反向传播计算梯度；但是每张卡在梯度计算后不会立刻进行更新，而是**先通过通信对梯度取平均**，**再通过梯度下降更新各自参数**，由此保证每张卡上模型参数仍然相同，在下次前向传播不同数据仍然面对相同的模型拷贝。

<div style="display:flex;flex-direction:row;justify-content:space-between; align-items:center" align="center">
  <div style="position:relative;width:50%;"><img src="images/data-parallel-forward.png" width="90%" align="center" /></div>
  <div style="position:relative;width:50%;"><img src="images/data-parallel-backward.png" width="90%" align="center" /></div>
</div>

&emsp; **代码实现** &ensp; CoLLiE 通过指定 **CollieConfig.dp_size 控制数据并行粒度**，**即有多少张显卡存储模型拷贝**；如下方代码所示，对于运行在4卡上的程序，设置CollieConfig.dp_size=4，就可以实现DP。

``` python
from collie.config import CollieConfig

config = CollieConfig.from_pretrained("fnlp/moss-moon-003-sft")

config.dp_size = 4
config.pp_size = 1
config.tp_size = 1
``` 

&emsp; **开销计算** &ensp; （待补充）

## 3.2 &ensp; 流水线并行（Pipeline Parallel）

&ensp; &ensp; 流水线并行（Pipeline Parallel，简称 PP），最早出自论文 [GPipe: Efficient Training of Giant Neural Networks using Pipeline Parallelism](https://arxiv.org/abs/1811.06965)。PP将模型横向切分，将参数分摊到不同的卡上，缓解了DP存储冗余的问题，但在时间开销上存在 bubble time 的问题，需要通过 micro_batch 和 1F1B 等技巧改进其初始效果。

&emsp; **前向传播** &ensp; 如下左图所示，同样是对于N层的大语言模型，K张显卡；**PP利用当前大模型由若干相同的自注意力层组成的特点**，**将模型按照层堆叠的方向切分**（左图只是演示，也可以多层对应一个切分），将不同切分分配到不同的显卡上。如此一来，每张显卡只存储模型的一部分，由此就充分利用了多张显卡的存储能力，扩充了服务器所能承载的模型参数上限，真正实现了模型的并行。在前向传播过程中，**每张卡只负责对应层的计算**，**以及将计算结果传输到存储下一层的显卡上**。

&emsp; **反向传播** &ensp; 和前向传播类似，PP在反向传播过程中，每张卡也只计算对应层的梯度，以及将梯度传输到存储上一层的显卡上，计算完梯度后即可以立刻更新参数。相比于DP需要在反向传播过程中，同步每层的梯度，**PP设备间的通信更小**；**但是PP设备间存在数据上的依赖**：前向传播时，顶层等待底层的计算结果，反向传播时，底层等待顶层的回传梯度，由此产生的闲置时间被称为 **bubble time**。

&ensp; &ensp; 由此，GPipe的作者提出，通过**将大的batch切成小份的micro_batch**，例如右上图所示，将1个batch变成4个microbatch。**在前向传播和反向传播两个阶段内部提升显卡的利用率**：第1个microbatch在第1个显卡上前向传播后，空出第1个显卡给第2个microbatch，同时在第2个显卡上继续计算；全部前向传播后，进行反向传播；第4个microbatch先在第4个显卡上反向传播，空出第4个显卡后给第3个microbatch，同时在第3个显卡上继续回传；**全部回传结束后**，**每个显卡内部对所有microbatch的梯度求平均再做更新**。

<div style="display:flex;flex-direction:row;justify-content:space-between; align-items:center" align="center">
  <div style="position:relative;width:36%;"><img src="images/pipeline-parallel.png" width="86%" align="center" /></div>
  <div style="position:relative;width:64%;"><img src="images/gpipe-1f1b.png" width="90%" align="left" /></div>
</div>

&emsp; **策略改进** &ensp; 如前文所述，PP的特点在于：一方面，横向切分，将不同的层放到不同的显卡上；另一方面，显卡之间存在数据依赖，只能依次执行，由此产生了 bubble time。为了进一步缓解 bubble time 的问题，后来的研究人员提出了 **1F1B 策略**，如右下图所示，这也是CoLLiE支持的PP实现方法。1F1B的核心思想是：对于每个microbatch，**执行一次前向传播后**，**立即执行一次反向传播**，但不立刻更新梯度；后一个microbatch利用前若干个microbatch前向/反向传播的空隙，进行前向/反向传播。

&ensp; &ensp; 观察右上图（GPipe）和右下图（1F1B）可以发现，一方面，上下两张图的bubble time是一样的，但是1F1B可以在相同的bubble time开销下，执行更多的microbatch，即 **1F1B相较于GPipe提升了显卡的利用率**，并且microbatch越多，1F1B在利用率上的优势更显著。另一方面，如虚线箭头所示，同样执行4个microbatch，1F1B并没有在GPipe之前完成，即 **1F1B相较于GPipe实际消耗时间是一样的**；GPipe只是提升了资源的利用率，在同等耗时内，执行了后续若干microbatch的计算过程。

&emsp; **代码实现** &ensp; CoLLiE 通过指定 **CollieConfig.pp_size 控制流水线并行粒度**，**即将模型横向切分为多少份**，并分别存储；如下方代码所示，对于运行在4卡上的程序，设置CollieConfig.pp_size=4，就可以实现PP。由于 PP每次梯度更新对应的数据条目数量 等于 microbatch大小 乘上 每次更新对应的microbatch数量，所以 对于PP **train_batch_size = train_micro_batch_size * gradient_accumulation_steps**。

&ensp; &ensp; 除此之外，参数 **CollieConfig.pp_partition_method 还可以控制PP切分方式**：默认切分方式 "parameter"，保证每张卡上参数量近似相等；"uniform"，保证每张卡上层数近似相等；"type:[regex]"，保证与"[regex]"正则匹配（不区分大小写）的层在每个切分上的数目近似相等，例如"type:transformer"使得每个切分上Transformer层的数目接近。

``` python
from collie.config import CollieConfig

config = CollieConfig.from_pretrained("fnlp/moss-moon-003-sft")

config.dp_size = 1
config.pp_size = 4
config.tp_size = 1

config.train_micro_batch_size = 2
config.gradient_accumulation_steps = 2
```

&emsp; **开销计算** &ensp; （待补充）

## 3.3 &ensp; 张量并行（Tensor Parallel）

&ensp; &ensp; 张量并行（Tensor Parallel，简称 TP），最早出自论文 [Megatron-LM: Training Multi-Billion Parameter Language Models Using Model Parallelism](https://arxiv.org/abs/1909.08053)。TP将模型横向切分，即让每张卡都具有模型每一层的每一切片；对于Transformer模型的每一部分，TP都给予了特定的切分方法，整体上大致遵循“先列切分，再行切分”的策略。

&emsp; **策略特点** &ensp; 纵向切分，mlp和selfattn的切法不同；mlp部分，gelu前的要split column-wise，gelu后的要split row-wise，可以想想象为multi-way mlp，避免切分后中间同步相加；selfattn部分，QKV要split column-wise，本质上就是multi-head，O要split row-wise，同样避免中间同步相加；其他部分：input_embed要split column-wise，切分出多个片段，output_embed要split row-wise，将多个片段聚合，但为了降低开销，只聚合loss。

相比于PP，需要server之间通信，TP尽量做到server内部通信；流水线并行PP和张量并行TP相互兼容，同时也兼容数据并行DP，合在一起就是3D并行 3D Parallel（TP本身根据切分维度也有2d-tp等，默认1d-tp）在batch_size较小时相较于流水线并行优势更加明显

<div style="position:relative;width:100%;" align="center">
  <img src="images/tensor-parallel.png" width="60%" align="center" />
</div>

&emsp; **前向传播** &ensp; 如何如何

&emsp; **反向传播** &ensp; 如何如何

&emsp; **代码实现** &ensp; 在CoLLiE中

``` python
from collie.config import CollieConfig

config = CollieConfig.from_pretrained("fnlp/moss-moon-003-sft")

config.dp_size = 1
config.pp_size = 1
config.tp_size = 4
```

&emsp; **开销计算** &ensp; （待补充）

## 3.4 &ensp; 整合：3D并行（3D Parallel）

&ensp; &ensp; DP、PP、TP 彼此之间是可以相互兼容促进的，三者的整合被称作**3D并行**。注：有些文献中直接将张量并行称为模型并行，在本教程中将流水线并行和张量并行都视为模型并行，即将模型参数分摊到各个不同的显卡上并行计算，的一种方法。

&ensp; &ensp; 提一嘴 activation checkpoint

<div style="position:relative;width:100%;" align="center">
  <img src="images/3d-parallel.png" width="60%" align="center" />
</div>

&emsp; **代码实现** &ensp; 在CoLLiE中；dp_size的默认值

``` python
from collie.config import CollieConfig

config = CollieConfig.from_pretrained("fnlp/moss-moon-003-sft")

config.dp_size = 2
config.pp_size = 2
config.tp_size = 2
``` 

&emsp; **开销计算** &ensp; （待补充）

## 3.5 &ensp; 零冗余优化器（ZeRO）  

&ensp; &ensp; 零冗余优化器（Zero Redundancy Optimizer，简称 ZeRO），最早出自论文 [ZeRO: Memory Optimizations Toward Training Trillion Parameter Models](https://arxiv.org/abs/1910.02054v3)，是数据并行和模型并行的结合，具体包括三个阶段，ZeRO-1至ZeRO-3，依次在数据并行的基础上，增加对于模型优化器状态、回传梯度、模型参数的切分。不同于PP和TP，ZeRO对每个模块直接切、均匀分。

&ensp; &ensp; ZeRO包括两个部分：ZeRO-DP，ZeRO-powered data parallel，ZeRO-R，负责优化冗余状态存储；其中，ZeRO-DP包括三个阶段：ZeRO1，切分优化器状态 state partitioning，ZeRO2，增加切分梯度 add gradient partitioning（以上相比数据并行不额外增加通信），ZeRO3，增加模型切分 add parameter partitioning

&ensp; &ensp; ZeRO3是数据并行和模型并行的结合（ZeRO不涉及具体的模型并行方式），每个卡负责不同的数据，并且负责不同的模型；数据前向计算到哪一层，就把那一层参数传过来，然后释放掉（如果那一层自己有就不用传输，不用释放）；损失反向回传到哪一层，就把那一层梯度传过来，释放前再返回那一层所在的卡将梯度聚合，并且根据那张卡上有的优化器状态，更新参数

&ensp; &ensp; 论及冗余状态存储（residual state memory，与model state memory相对，即参数 梯度 优化器的存储），ZeRO-R涉及三个方面：激活值 activation，activation checkpointing & partitioning；缓存区 temporary buffers，定长缓存区；碎片管理 memory fragmentation，反向传播时，激活值的checkpoint和参数的梯度存储较长，重计算的激活值和激活值的梯度存储较短，前者放在连续缓存区，避免和后者交叉使用产生碎片

<div style="display:flex;flex-direction:row;justify-content:space-between; align-items:center" align="center">
  <div style="position:relative;width:50%;"><img src="images/zero-forward.png" width="90%" align="center" /></div>
  <div style="position:relative;width:50%;"><img src="images/zero-backward.png" width="90%" align="center" /></div>
</div>

&ensp; &ensp; 在CoLLiE中

``` python
from collie.config import CollieConfig

config = CollieConfig.from_pretrained("fnlp/moss-moon-003-sft")

config.dp_size = 1  # 4
config.pp_size = 1
config.tp_size = 1

config.ds_config = {
    "fp16": {
        "enabled": True
    },
    "zero_allow_untested_optimizer": True,
    "zero_force_ds_cpu_optimizer": False,
    "zero_optimization": {
        "stage": 3,
        "offload_optimizer": {
            "device": "cpu",
            "pin_memory": False
        }
    }
}
``` 

&emsp; **开销计算** &ensp; （待补充）